In [ ]:
# 1. INSTALL & IMPORTS
!pip install ultralytics kagglehub gradio -q
import os
import shutil
import kagglehub
import gradio as gr
from sklearn.model_selection import train_test_split
from ultralytics import YOLO

# 2. DATASET DOWNLOAD & PREPARATION
print("--- Step 1: Downloading & Organizing Data ---")
path = kagglehub.dataset_download("alxmamaev/flowers-recognition")
raw_source = os.path.join(path, 'flowers') if os.path.exists(os.path.join(path, 'flowers')) else path

# Set up YOLO folder structure
base_dir = '/content/flower_data'
classes = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']
for split in ['train', 'val']:
    for flower in classes:
        os.makedirs(os.path.join(base_dir, split, flower), exist_ok=True)

# Split images 80/20
for flower in classes:
    flower_dir = os.path.join(raw_source, flower)
    if not os.path.isdir(flower_dir): continue
    imgs = [f for f in os.listdir(flower_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    train_imgs, val_imgs = train_test_split(imgs, test_size=0.2, random_state=42)

    for img in train_imgs: shutil.copy(os.path.join(flower_dir, img), os.path.join(base_dir, 'train', flower, img))
    for img in val_imgs: shutil.copy(os.path.join(flower_dir, img), os.path.join(base_dir, 'val', flower, img))

# 3. MODEL TRAINING
print("--- Step 2: Training Model (this may take a few minutes) ---")
model = YOLO('yolov8n-cls.pt')
model.train(data=base_dir, epochs=10, imgsz=224, batch=32, name='flower_model')

# 4. GRADIO INTERFACE (THE FRONTEND)
print("--- Step 3: Launching Interactive Interface ---")
def predict_flower(img):
    results = model.predict(source=img, conf=0.25)
    probs = results[0].probs.data.tolist()
    names = results[0].names
    return {names[i]: probs[i] for i in range(len(names))}

demo = gr.Interface(
    fn=predict_flower,
    inputs=gr.Image(type="pil", label="Upload Flower"),
    outputs=gr.Label(num_top_classes=3, label="Top Predictions"),
    title="🌸 Flower Recognition AI",
    description="Trained on Daisy, Dandelion, Rose, Sunflower, and Tulip."
)

# share=True creates a public URL you can send to anyone
demo.launch(share=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.9 MB/s eta 0:00:00
